In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/CV_seminar_project

/content/drive/MyDrive/CV_seminar_project


In [3]:
import torch
state_dict= torch.load('/content/drive/MyDrive/CV_seminar_project/resnet50_46_94.977.pth')


학습한 가중치 불러오기

In [5]:
best_epoch = state_dict['epoch']
best_val_acc = state_dict['val_acc']
weight=state_dict['net']

학습한 가중치 적용하기

In [6]:
from torchvision import models # 모델 라이브러리 함수
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu' # device 배정 (gpu 사용 가능하면 cuda 아니면 cpu 사용)
torch.manual_seed(42)
if device == 'cuda':
  torch.cuda.manual_seed_all(42)


resnet_50 = models.resnet50(pretrained=False).to(device) # 선행학습 여부

# finetuning
import torch.nn as nn # 파이토치 뉴럴네트워크 layer 라이브러리
resnet_50.fc = nn.Linear(resnet_50.fc.in_features, 3).to(device)

#가중치 적용
resnet_50.load_state_dict(weight)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

> test loder 구현

In [12]:
from dataset import Custom_dataset as C #dataset만드는 class
from torch.utils.data import Dataset, DataLoader
import cv2
import os 
import torch
import torchvision
from torchvision import transforms # 이미지 데이터 augmentation
import glob
import albumentations as A #augumentation 패키지
from albumentations.pytorch.transforms import ToTensorV2 # albumentations 텐서화 함수

root_path = '/content/drive/MyDrive/CV_seminar_project'
batch_size=8

test_transforms = A.Compose([
    A.Resize(224,224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0), # 텐서타입은 안해줌
    ToTensorV2() # Normalize를 먼저하고 tensor화를 진행해야한다.
])

### Pytorch 데이터 클래스 생성
test_class = C(root_path=root_path, mode='test', transform=test_transforms)

### Pytorch BatchLoader 생성 (학습에 이용할 최종 dataloader)
from torch.utils.data import DataLoader as DataLoader

data_loader = DataLoader(test_class, batch_size=batch_size, shuffle = False, num_workers=0)

하이퍼파라미터 초기화

In [14]:
import torch.nn as nn

# 하이퍼 파라미터
batch_size = 8
optimizer_name = 'adam'
model_name = 'resnet50'
criterion = nn.CrossEntropyLoss().to(device) # cost function

test셋에 학습된 모델 적용

In [15]:
import numpy as np

test_acc_lst = []
test_loss_lst = []

test_running_loss = 0.0
test_acc = 0

total = 0
correct = 0
resnet_50.eval() # 평가모드 / self.evla() == self.train(False)


#.eval() 함수는 evaluation 과정에서 사용하지 않아야 하는 layer들을 알아서 off 
with torch.no_grad():
  for ii, (test_img, test_label) in enumerate(data_loader):
    # gpu에 할당
    test_img = test_img.to(device)
    test_label = test_label.to(device)

    output = resnet_50(test_img) # 모델에 입력
    loss = criterion(output, test_label) #CrossEntropyLoss() 적용

    # loss & acc
    test_running_loss += loss.item()
    _, predictions = torch.max(output.data ,dim = 1 )
    
    total += test_label.size(0)
    correct += (predictions == test_label).sum().item()
    test_acc += 100 * (correct/total)
    

test_running_loss = round(test_running_loss/(ii+1), 3) # 소수점 반올림 / ii는 iteration (배치사이즈로 데이터를 몇번 나누어 계산했는가)
test_acc = round(test_acc/(ii+1), 3)
print(f'resnet50_46_94.977 Loss : {test_running_loss}, Accuracy : {test_acc}% \n')

resnet50_46_94.977 Loss : 0.302, Accuracy : 87.116% 

